In [0]:
from pyspark.sql import functions as F

base_path = "/tmp/dbx_demo/nyc_taxi_delta"
#tpep_pickup_datetime:timestamp
#tpep_dropoff_datetime:timestamp
#trip_distance:double
#fare_amount:double
#pickup_zip:integer
#dropoff_zip:integer

# Load sample Delta table
df = spark.read.format("delta").load("dbfs:/databricks-datasets/nyctaxi-with-zipcodes/subsampled")

clean = (
    df.select(
        "tpep_pickup_datetime",
        "tpep_dropoff_datetime",
        "trip_distance",
        "fare_amount",
        "pickup_zip",
        "dropoff_zip",
    )
    .filter(
        (F.col("fare_amount") > 0) &
        (F.col("trip_distance") > 0)
    )
)

clean.write.format("delta").mode("overwrite").save(base_path)

spark.sql("CREATE DATABASE IF NOT EXISTS dbx_demo")
spark.sql("DROP TABLE IF EXISTS dbx_demo.nyc_taxi")
spark.sql(f"CREATE TABLE dbx_demo.nyc_taxi USING DELTA LOCATION '{base_path}'")

display(spark.table("dbx_demo.nyc_taxi").limit(10))